In [1]:
from pulp import *   # PuLP - Python Library for Linear Programming
import pandas as pd
import numpy as np

In [2]:
n_supply = 100   #number of supply points
n_demand = 4096  #number of demand points

#Importing csv files 
demand_set = pd.read_csv('Demand_Predictions.csv') 
supply_set = pd.read_csv('exisiting_EV_infrastructure_2019.csv')
dist_mat=pd.read_csv('Distance_Matrix.csv')

In [3]:
demand_set

,demand_point_index,x_coordinate,y_coordinate,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,0.5,0.5,0.352242,0.667932,0.958593,2.911901,4.338274,6.561995,8.454417,10.595324,13.119572,15.088540,16.925371
1,1,1.5,0.5,0.325940,0.591964,0.862652,2.589068,4.196034,5.745551,8.753195,11.126995,12.020091,13.453829,13.827596
2,2,2.5,0.5,0.373752,0.591890,0.969733,2.641432,3.541772,5.469161,8.414627,10.115336,14.018254,17.501605,21.635659
3,3,3.5,0.5,0.420686,0.584055,0.906547,2.378577,3.888121,5.846089,9.083868,12.424885,15.012302,18.674598,22.179762
4,4,4.5,0.5,0.475621,0.647940,0.981544,2.665400,4.218711,6.776609,8.851107,11.731131,16.355563,20.526166,25.623670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4091,4091,59.5,63.5,0.171015,0.334565,0.556055,1.373291,1.837586,2.517146,3.352280,4.149888,5.426193,6.540436,7.850964
4092,4092,60.5,63.5,0.041716,0.061741,0.131291,0.386540,0.755846,0.941116,1.107797,1.309479,2.057450,2.459605,3.065697
4093,4093,61.5,63.5,0.100895,0.180352,0.296299,0.705373,1.300220,1.608609,1.822806,2.333681,3.218519,3.785937,4.543876
4094,4094,62.5,63.5,0.155353,0.290825,0.557803,1.516066,2.399426,2.719197,4.494515,6.096858,6.262574,7.283772,7.752700


In [4]:
supply_set

,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS
0,0,50.163110,19.412014,23,5,17
1,1,37.336451,58.119225,27,4,7
2,2,46.709232,57.525650,31,6,14
3,3,30.528626,55.379835,26,5,5
4,4,51.521781,35.116755,32,11,6
...,...,...,...,...,...,...
95,95,45.471204,20.999414,24,3,9
96,96,30.318396,33.388335,32,5,21
97,97,36.218839,22.235766,32,4,25
98,98,42.936915,38.122442,28,7,5


In [5]:
dist_mat=dist_mat.to_numpy()

In [6]:
col=[]
for i in range(supply_set.shape[0]):
    ans=supply_set.iloc[i:i+1,4].values*200+supply_set.iloc[i:i+1,5].values*400
    col.append(int(ans))
col=np.array(col)
supply_set['Smax19']=col

In [7]:
#Demand values of 2020
demands20=np.array(demand_set['2020'])

In [8]:
model = LpProblem("Cost-Optimization-Model", LpMinimize)

In [9]:
index_numbers_x = [str(i) for i in range(n_supply*n_demand)]
index_numbers_y = [str(i) for i in range(n_supply)]
index_numbers_x.sort()
index_numbers_y.sort()

In [10]:
DV_variables = LpVariable.matrix("X", index_numbers_x, cat = "Continuous", lowBound= 0 )
dsij_mat20 = np.array(DV_variables).reshape(n_supply,n_demand)

In [11]:
Smax20=[]
park_slot=supply_set['total_parking_slots']-(supply_set['existing_num_SCS']+supply_set['existing_num_FCS'])
park_slot.to_numpy()
for i in range(n_supply):
    lb=int(supply_set.iloc[i:i+1,6].values)
    ub=lb+(park_slot[i]*400)
    temp=LpVariable('Y_{}'.format(index_numbers_y[i]),cat='Integer',lowBound=lb,upBound=ub)
    Smax20.append(temp)
Smax20=np.array(Smax20)

In [12]:
cost_eq1 = lpSum(dsij_mat20*dist_mat)

In [13]:
S_var=[]
F_var=[]

index_numbers_c = [str(i) for i in range(n_supply)]
index_numbers_c.sort()

for i in range(n_supply):
    lb=int(supply_set.iloc[i:i+1,4].values)
    temp=LpVariable('S_{}'.format(index_numbers_c[i]),cat='Integer', lowBound=lb)
    S_var.append(temp)
S_var=np.array(S_var)

for i in range(n_supply):
    lb=int(supply_set.iloc[i:i+1,5].values)
    temp=LpVariable('F_{}'.format(index_numbers_c[i]),cat='Integer',lowBound=lb)
    F_var.append(temp)
F_var=np.array(F_var)

In [14]:
cost_eq2=lpSum(S_var)+1.5*lpSum(F_var)

In [15]:
cost_eq3=cost_eq1+(600*cost_eq2)

In [16]:
type(cost_eq3)

pulp.pulp.LpAffineExpression

In [17]:
model += cost_eq3

model

In [18]:
#Supply Constraint
#Column summation must be less than or equal to Smax 
for i in range(n_supply):
    model += lpSum(dsij_mat20[i][j] for j in range(n_demand)) <= Smax20[i]-0.01 , "Supply Constraints " + str(i)

In [19]:
# Demand Constraints
#Row summation must be equal to demand forecast
for j in range(n_demand):
    model += lpSum(dsij_mat20[i][j] for i in range(n_supply)) >= demands20[j] , "Demand Constraints " + str(j)

In [20]:
#Supply Constraint
for i in range(n_supply):
    constraint = (S_var[i]*200)+(F_var[i]*400) >= Smax20[i]
    model += constraint

In [21]:
model.solve(PULP_CBC_CMD())
status =  LpStatus[model.status]
print(status)

Optimal


In [22]:
print("Total Cost:", model.objective.value())

Total Cost: 2883550.3720495747


# DSij Variable's value
for v in model.variables():
    print(v.name,"=", v.value())

In [23]:
allVar=[]
for v in model.variables():
    k=v.value()
    allVar.append(k)
#allVar

In [24]:
p=0
F_var=[]
S_var=[]
for i in range(n_supply):
    F_var.append(int(allVar[p]))
    p+=1
for i in range(n_supply):
    S_var.append(int(allVar[p]))
    p+=1
#print(F_var)
#print(S_var)

In [25]:
dsij_mat20=np.zeros((100,4096))
    
for i in range(n_supply):
    for j in range(n_demand):
        dsij_mat20[i][j]=allVar[p]
        p+=1
#dsij_mat20

In [26]:
Smax20=[]
for i in range(n_supply):
    Smax20.append(int(allVar[p]))
    p+=1
Smax20=np.array(Smax20)
#Smax20

In [27]:
dsij_mat20=pd.DataFrame(dsij_mat20)
dsij_mat20.to_csv('Total_Dsij_20_v2.csv')

In [28]:
#Checking the Demand constraint (row summation)
print(sum(dsij_mat20[0]))
#Comparing with demand value
print(demands20[0])

16.925371
16.925370693206787


In [29]:
#Checking the Supply constraint (column summation)
print(sum(dsij_mat20.iloc[0,:].values))
#Comparing with supply value
print(Smax20[0])

8199.989997300001
8200


supply_set

In [30]:
supply_set['Smax20']=Smax20
#supply_set

In [31]:
supply_set['SCS_20']=S_var
supply_set['FCS_20']=F_var
supply_set

,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS,Smax19,Smax20,SCS_20,FCS_20
0,0,50.163110,19.412014,23,5,17,7800,8200,5,18
1,1,37.336451,58.119225,27,4,7,3600,3600,4,7
2,2,46.709232,57.525650,31,6,14,6800,6800,6,14
3,3,30.528626,55.379835,26,5,5,3000,3000,5,5
4,4,51.521781,35.116755,32,11,6,4600,4600,11,6
...,...,...,...,...,...,...,...,...,...,...
95,95,45.471204,20.999414,24,3,9,4200,7800,3,18
96,96,30.318396,33.388335,32,5,21,9400,11000,5,25
97,97,36.218839,22.235766,32,4,25,10800,12000,4,28
98,98,42.936915,38.122442,28,7,5,3400,3400,7,5


In [32]:
supply_set.to_csv('Total_infra_20_v2.csv')